In [1]:
import  neo4j
import pandas as pd
import pyodbc

In [3]:


server = 'mcruebs04.isad.isadroot.ex.ac.uk'
database = 'BEMM459_GroupO'
username = 'GroupO'
password = 'MrjV827*Wr'

try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};',
        autocommit=True
    )
    print("Connected to Microsoft SQL Server!")
except Exception as e:
    print("Error connecting to SQL Server:", e)


Connected to Microsoft SQL Server!


In [ ]:
cursor = conn.cursor()



# Create tables
tables = [
"""  
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Email VARCHAR(100),
    Phone VARCHAR(20),
    Address VARCHAR(200),
    Type VARCHAR(50))
"""

"""
CREATE TABLE Parcels (
    ParcelID INT PRIMARY KEY,
    ParcelName VARCHAR(100),
    CustomerID INT,
    Weight FLOAT,
    Type VARCHAR(50),
    ShippingMethod VARCHAR(50),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
)"""

 """
CREATE TABLE Shipments (
    ShipmentID INT PRIMARY KEY,
    ShipmentName VARCHAR(100),
    ParcelID INT,
    ShipmentDate DATETIME,
    DeliveryDate DATETIME,
    Status VARCHAR(50),
    CurrentLocation VARCHAR(100),
    FOREIGN KEY (ParcelID) REFERENCES Parcels(ParcelID)
)"""

"""
CREATE TABLE Personnel (
    PersonnelID INT PRIMARY KEY,
    Name VARCHAR(100),
    Phone VARCHAR(20),
    Role VARCHAR(50)
)"""

"""
CREATE TABLE ShipmentPersonnel (
    ShipmentID INT,
    PersonnelID INT,
    FOREIGN KEY (ShipmentID) REFERENCES Shipments(ShipmentID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID)
);"""

"""
CREATE TABLE Analytics (
    ShipmentID INT,
    Delays INT,
    Efficiency FLOAT,
    CustomerRating INT,
    FOREIGN KEY (ShipmentID) REFERENCES Shipments(ShipmentID)
)"""
        
]

for table in tables:
    try:
        cursor.execute(table)
        print("Table created successfully!")
    except Exception as e:
        print("Error creating table:", e)

conn.commit()
cursor.close()
conn.close()


In [ ]:
import pandas as pd
import pyodbc

# SQL Data Loader
class SQLDataLoader:
    def __init__(self, db_url):
        self.conn = pyodbc.connect(db_url)

    def load_data(self, file_path, sheet_name, table_name):
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        cursor = self.conn.cursor()
        for _, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(query, tuple(row))
        self.conn.commit()
        cursor.close()
        print(f"Data loaded into {table_name} successfully!")

# Usage 
sql_loader = SQLDataLoader("DRIVER={ODBC Driver 17 for SQL Server};SERVER=mcruebs04.isad.isadroot.ex.ac.uk;DATABASE=BEMM459_GroupO;UID=GroupO;PWD=MrjV827*Wr")



In [ ]:
# Load data into each table
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Customers", "Customers")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Parcels", "Parcels")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Shipments", "Shipments")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Personnel", "Personnel")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "ShipmentPersonnel", "ShipmentPersonnel")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Analytics", "Analytics")


Querying to check if the data is loaded correctly

In [18]:

cursor = sql_loader.conn.cursor()

tables = ["Customers", "Parcels", "Shipments", "Personnel", "ShipmentPersonnel", "Analytics"]

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"Table {table} has {count} rows.")




for table in tables:
    cursor.execute(f"SELECT TOP 5 * FROM {table}")
    rows = cursor.fetchall()
    print(f"\nPreview of {table}:")
    for row in rows:
        print(row)
cursor.close()


Table Customers has 83 rows.
Table Parcels has 269 rows.
Table Shipments has 269 rows.
Table Personnel has 27 rows.
Table ShipmentPersonnel has 556 rows.
Table Analytics has 269 rows.

Preview of Customers:
(1, 'Customer_1', 'customer1@example.com', '+44 7126855092', 'Address_1', 'Premium')
(2, 'Customer_2', 'customer2@example.com', '+44 7362950628', 'Address_2', 'Regular')
(3, 'Customer_3', 'customer3@example.com', '+44 7249827706', 'Address_3', 'Regular')
(4, 'Customer_4', 'customer4@example.com', '+44 7826600539', 'Address_4', 'Regular')
(5, 'Customer_5', 'customer5@example.com', '+44 7734036506', 'Address_5', 'Premium')

Preview of Parcels:
(1, 'Parcel_1', 68, 18.41062314071619, 'Perishable', 'Express')
(2, 'Parcel_2', 3, 2.681420643757454, 'Perishable', 'Express')
(3, 'Parcel_3', 31, 1.629540746636657, 'Fragile', 'Standard')
(4, 'Parcel_4', 63, 16.41245344169172, 'Solid', 'Standard')
(5, 'Parcel_5', 61, 18.96354925424732, 'Solid', 'Express')

Preview of Shipments:
(1, 'Shipment_1'

In [20]:

cursor = sql_loader.conn.cursor()
#Function to get column names and data types
def get_column_info(table_name):
    query = f"""
    SELECT COLUMN_NAME, DATA_TYPE
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = '{table_name}';
    """
    cursor.execute(query)
    print(f"\nColumns for table '{table_name}':")
    for row in cursor.fetchall():
        print(f"Column Name: {row.COLUMN_NAME}, Data Type: {row.DATA_TYPE}")

# Loop through each table and print the column details
for table in tables:
    get_column_info(table)

# Close connection
cursor.close()


Columns for table 'Customers':
Column Name: CustomerID, Data Type: int
Column Name: Name, Data Type: varchar
Column Name: Email, Data Type: varchar
Column Name: Phone, Data Type: varchar
Column Name: Address, Data Type: varchar
Column Name: Type, Data Type: varchar

Columns for table 'Parcels':
Column Name: ParcelID, Data Type: int
Column Name: ParcelName, Data Type: varchar
Column Name: CustomerID, Data Type: int
Column Name: Weight, Data Type: float
Column Name: Type, Data Type: varchar
Column Name: ShippingMethod, Data Type: varchar

Columns for table 'Shipments':
Column Name: ShipmentID, Data Type: int
Column Name: ShipmentName, Data Type: varchar
Column Name: ParcelID, Data Type: int
Column Name: ShipmentDate, Data Type: datetime
Column Name: DeliveryDate, Data Type: datetime
Column Name: Status, Data Type: varchar
Column Name: CurrentLocation, Data Type: varchar

Columns for table 'Personnel':
Column Name: PersonnelID, Data Type: int
Column Name: Name, Data Type: varchar
Column 

In [ ]:
Neo4j setup

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

# Connection details
URI = "neo4j+s://dd0c088b.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = "lapDEOAyHjUWt31Bg123clw9s6949mg56APvryc4-tY"

driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
# Load the Excel file
file_path = r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx"
excel_data = pd.ExcelFile(file_path)

# Function to create a node
def create_node(tx, label, **properties):
    query = f"MERGE (n:{label} {{{', '.join(f'{key}: ${key}' for key in properties.keys())}}})"
    tx.run(query, **properties)

# Function to create a relationship
def create_relationship(tx, label1, id1, label2, id2, relationship):
    query = f"""
    MATCH (a:{label1} {{ID: $id1}}), (b:{label2} {{ID: $id2}})
    MERGE (a)-[:{relationship}]->(b)
    """
    tx.run(query, id1=id1, id2=id2)

# Insert data from each sheet with relationships
def load_data():
    with driver.session() as session:
        # Load Customers
        customers = pd.read_excel(file_path, sheet_name='Customers')
        for _, row in customers.iterrows():
            session.write_transaction(create_node, "Customer", ID=str(row['CustomerID']), Name=row['Name'], ContactInfo=row['Email'], Type=row['Type'])

        # Load Personnel
        personnel = pd.read_excel(file_path, sheet_name='Personnel')
        for _, row in personnel.iterrows():
            session.write_transaction(create_node, "Personnel", ID=str(row['PersonnelID']), Name=row['Name'], Role=row['Role'])

        # Load Parcels and create OWNS relationship
        parcels = pd.read_excel(file_path, sheet_name='Parcels')
        for _, row in parcels.iterrows():
            session.write_transaction(create_node, "Parcel", ID=str(row['ParcelID']), Name=row['ParcelName'], Weight=row['Weight'], Type=row['Type'])
            session.write_transaction(create_relationship, "Customer", str(row['CustomerID']), "Parcel", str(row['ParcelID']), "OWNS")

        # Load Shipment Statuses as separate nodes
        shipments = pd.read_excel(file_path, sheet_name='Shipments')
        for _, row in shipments.iterrows():
            # Create a status node for each unique status
            session.write_transaction(create_node, "Status", ID=str(row['ShipmentID']), Name=row['Status'])
            # Create a relationship between Parcel and its Status
            session.write_transaction(create_relationship, "Parcel", str(row['ParcelID']), "Status", str(row['ShipmentID']), "DELIVERY_STATUS")
            # Create a shipment node as well
            session.write_transaction(create_node, "Shipment", ID=str(row['ShipmentID']), Name=row['ShipmentName'], Dates=str(row['ShipmentDate']), Location=row['CurrentLocation'])

        # Load Analytics and create HAS_ANALYTICS relationship
        analytics = pd.read_excel(file_path, sheet_name='Analytics')
        for _, row in analytics.iterrows():
            session.write_transaction(create_node, "Analytics", ID=str(row['ShipmentID']), Delays=row['Delays'], Efficiency=row['Efficiency'], CustomerRating=row['CustomerRating'])
            session.write_transaction(create_relationship, "Shipment", str(row['ShipmentID']), "Analytics", str(row['ShipmentID']), "HAS_ANALYTICS")

        # Load ShipmentPersonnel relationships
        shipment_personnel = pd.read_excel(file_path, sheet_name='ShipmentPersonnel')
        for _, row in shipment_personnel.iterrows():
            session.write_transaction(create_relationship, "Shipment", str(row['ShipmentID']), "Personnel", str(row['PersonnelID']), "HANDLED_BY")

    print("Data loaded successfully!")

# Run the data loading function
load_data()

driver.close()


C:\Users\bgane\AppData\Local\Temp\ipykernel_41240\4198792644.py:33: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Customer", ID=str(row['CustomerID']), Name=row['Name'], ContactInfo=row['Email'], Type=row['Type'])
C:\Users\bgane\AppData\Local\Temp\ipykernel_41240\4198792644.py:38: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Personnel", ID=str(row['PersonnelID']), Name=row['Name'], Role=row['Role'])
C:\Users\bgane\AppData\Local\Temp\ipykernel_41240\4198792644.py:43: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Parcel", ID=str(row['ParcelID']), Name=row['ParcelName'], Weight=row['Weight'], Type=row['Type'])
C:\Users\bgane\AppData\Local\Temp\ipykernel_41240\4198792644.py:44: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relat

Data loaded successfully!


Application backEnd code


Run this first then save the frontend python file in the same directory/folder as frontend.py and run the following command in the terminal:


streamlit run frontend.py

In [ ]:

from fastapi import FastAPI, HTTPException, Depends, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
import pyodbc
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime

app = FastAPI(title="DHL Logistics Backend API")

# Add CORS middleware to allow frontend to connect
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

# SQL Server Connection
def get_sql_connection():
    server = 'mcruebs04.isad.isadroot.ex.ac.uk'
    database = 'BEMM459_GroupO'
    username = 'GroupO'
    password = 'MrjV827*Wr'
    
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};',
        autocommit=True
    )
    try:
        yield conn
    finally:
        conn.close()

# Neo4j Connection
def get_neo4j_driver():
    uri = "neo4j+s://dd0c088b.databases.neo4j.io"
    username = "neo4j"
    password = "lapDEOAyHjUWt31Bg123clw9s6949mg56APvryc4-tY"
    
    driver = GraphDatabase.driver(uri, auth=(username, password))
    try:
        yield driver
    finally:
        driver.close()

# Pydantic models for request/response validation
class Customer(BaseModel):
    CustomerID: int
    Name: str
    Email: str
    Phone: str
    Address: str
    Type: str

class Parcel(BaseModel):
    ParcelID: int
    ParcelName: str
    CustomerID: int
    Weight: float
    Type: str
    ShippingMethod: str

class Shipment(BaseModel):
    ShipmentID: int
    ShipmentName: str
    ParcelID: int
    ShipmentDate: datetime
    DeliveryDate: datetime
    Status: str
    CurrentLocation: str

class Personnel(BaseModel):
    PersonnelID: int
    Name: str
    Phone: str
    Role: str

class Analytics(BaseModel):
    ShipmentID: int
    Delays: int
    Efficiency: float
    CustomerRating: int

class ShipmentPersonnelLink(BaseModel):
    ShipmentID: int
    PersonnelID: int

class StatusUpdate(BaseModel):
    Status: str

class NodeInfo(BaseModel):
    Label: str
    ID: str
    Properties: Dict[str, Any]

class RelationshipInfo(BaseModel):
    StartNode: str
    EndNode: str
    Type: str
    Properties: Optional[Dict[str, Any]] = {}

# Health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "healthy", "timestamp": datetime.now().isoformat()}

# SQL Server Endpoints

# Customers
@app.get("/customers", response_model=List[Customer])
async def get_customers(
    conn: pyodbc.Connection = Depends(get_sql_connection),
    limit: int = Query(10, description="Maximum number of customers to return"),
    offset: int = Query(0, description="Number of customers to skip")
):
    cursor = conn.cursor()
    try:
        cursor.execute(f"SELECT * FROM Customers ORDER BY CustomerID OFFSET {offset} ROWS FETCH NEXT {limit} ROWS ONLY")
        columns = [column[0] for column in cursor.description]
        
        result = []
        for row in cursor.fetchall():
            result.append(dict(zip(columns, row)))
        
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.get("/customers/{customer_id}", response_model=Customer)
async def get_customer(customer_id: int, conn: pyodbc.Connection = Depends(get_sql_connection)):
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT * FROM Customers WHERE CustomerID = ?", (customer_id,))
        columns = [column[0] for column in cursor.description]
        
        row = cursor.fetchone()
        if not row:
            raise HTTPException(status_code=404, detail=f"Customer with ID {customer_id} not found")
        
        return dict(zip(columns, row))
    except Exception as e:
        if "not found" in str(e):
            raise
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.post("/customers", response_model=Customer)
async def create_customer(customer: Customer, conn: pyodbc.Connection = Depends(get_sql_connection)):
    cursor = conn.cursor()
    try:
        cursor.execute(
            "INSERT INTO Customers (CustomerID, Name, Email, Phone, Address, Type) VALUES (?, ?, ?, ?, ?, ?)",
            (customer.CustomerID, customer.Name, customer.Email, customer.Phone, customer.Address, customer.Type)
        )
        return customer
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

# Parcels
@app.get("/parcels", response_model=List[Parcel])
async def get_parcels(
    conn: pyodbc.Connection = Depends(get_sql_connection),
    limit: int = Query(10, description="Maximum number of parcels to return"),
    offset: int = Query(0, description="Number of parcels to skip"),
    customer_id: Optional[int] = Query(None, description="Filter by customer ID")
):
    cursor = conn.cursor()
    try:
        query = "SELECT * FROM Parcels"
        params = []
        
        if customer_id:
            query += " WHERE CustomerID = ?"
            params.append(customer_id)
            
        query += f" ORDER BY ParcelID OFFSET {offset} ROWS FETCH NEXT {limit} ROWS ONLY"
        
        cursor.execute(query, params)
        columns = [column[0] for column in cursor.description]
        
        result = []
        for row in cursor.fetchall():
            result.append(dict(zip(columns, row)))
        
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.get("/parcels/{parcel_id}", response_model=Parcel)
async def get_parcel(parcel_id: int, conn: pyodbc.Connection = Depends(get_sql_connection)):
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT * FROM Parcels WHERE ParcelID = ?", (parcel_id,))
        columns = [column[0] for column in cursor.description]
        
        row = cursor.fetchone()
        if not row:
            raise HTTPException(status_code=404, detail=f"Parcel with ID {parcel_id} not found")
        
        return dict(zip(columns, row))
    except Exception as e:
        if "not found" in str(e):
            raise
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.put("/parcels/{parcel_id}", response_model=Parcel)
async def update_parcel(
    parcel_id: int, 
    parcel_update: Parcel, 
    conn: pyodbc.Connection = Depends(get_sql_connection)
):
    cursor = conn.cursor()
    try:
        # Check if parcel exists
        cursor.execute("SELECT * FROM Parcels WHERE ParcelID = ?", (parcel_id,))
        if not cursor.fetchone():
            raise HTTPException(status_code=404, detail=f"Parcel with ID {parcel_id} not found")
        
        # Update parcel
        cursor.execute(
            """
            UPDATE Parcels 
            SET ParcelName = ?, CustomerID = ?, Weight = ?, Type = ?, ShippingMethod = ?
            WHERE ParcelID = ?
            """,
            (
                parcel_update.ParcelName, 
                parcel_update.CustomerID, 
                parcel_update.Weight, 
                parcel_update.Type, 
                parcel_update.ShippingMethod, 
                parcel_id
            )
        )
        
        return parcel_update
    except Exception as e:
        if "not found" in str(e):
            raise
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

# Shipments
@app.get("/shipments", response_model=List[Shipment])
async def get_shipments(
    conn: pyodbc.Connection = Depends(get_sql_connection),
    limit: int = Query(10, description="Maximum number of shipments to return"),
    offset: int = Query(0, description="Number of shipments to skip"),
    status: Optional[str] = Query(None, description="Filter by status"),
    customer_id: Optional[int] = Query(None, description="Filter by customer ID")
):
    cursor = conn.cursor()
    try:
        query = """
            SELECT s.* FROM Shipments s
        """
        params = []
        
        if customer_id:
            query += """
                JOIN Parcels p ON s.ParcelID = p.ParcelID
                WHERE p.CustomerID = ?
            """
            params.append(customer_id)
            
            if status:
                query += " AND s.Status = ?"
                params.append(status)
        elif status:
            query += " WHERE s.Status = ?"
            params.append(status)
            
        query += f" ORDER BY s.ShipmentID OFFSET {offset} ROWS FETCH NEXT {limit} ROWS ONLY"
        
        cursor.execute(query, params)
        columns = [column[0] for column in cursor.description]
        
        result = []
        for row in cursor.fetchall():
            result.append(dict(zip(columns, row)))
        
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.get("/shipments/{shipment_id}", response_model=Shipment)
async def get_shipment(shipment_id: int, conn: pyodbc.Connection = Depends(get_sql_connection)):
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT * FROM Shipments WHERE ShipmentID = ?", (shipment_id,))
        columns = [column[0] for column in cursor.description]
        
        row = cursor.fetchone()
        if not row:
            raise HTTPException(status_code=404, detail=f"Shipment with ID {shipment_id} not found")
        
        return dict(zip(columns, row))
    except Exception as e:
        if "not found" in str(e):
            raise
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.put("/shipments/{shipment_id}/status", response_model=dict)
async def update_shipment_status(
    shipment_id: int, 
    status_update: StatusUpdate, 
    conn: pyodbc.Connection = Depends(get_sql_connection),
    neo4j_driver: GraphDatabase.driver = Depends(get_neo4j_driver)
):
    # Update SQL database
    cursor = conn.cursor()
    try:
        # Check if shipment exists
        cursor.execute("SELECT * FROM Shipments WHERE ShipmentID = ?", (shipment_id,))
        if not cursor.fetchone():
            raise HTTPException(status_code=404, detail=f"Shipment with ID {shipment_id} not found")
        
        # Update shipment status
        cursor.execute(
            "UPDATE Shipments SET Status = ? WHERE ShipmentID = ?",
            (status_update.Status, shipment_id)
        )
        
    except Exception as e:
        if "not found" in str(e):
            raise
        raise HTTPException(status_code=500, detail=f"SQL Database error: {str(e)}")
    finally:
        cursor.close()
    
    # Update Neo4j database
    try:
        with neo4j_driver.session() as session:
            # Update the Status node
            result = session.run(
                """
                MATCH (s:Status {ID: $shipment_id})
                SET s.Name = $status
                RETURN s
                """,
                shipment_id=str(shipment_id),
                status=status_update.Status
            )
            
            # Check if the node was found and updated
            if not result.single():
                # Create the node if it doesn't exist
                session.run(
                    """
                    MERGE (s:Status {ID: $shipment_id, Name: $status})
                    """,
                    shipment_id=str(shipment_id),
                    status=status_update.Status
                )
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Neo4j Database error: {str(e)}")
    
    return {"message": "Shipment status updated successfully", "status": status_update.Status}

# Analytics
@app.get("/analytics", response_model=List[Analytics])
async def get_analytics(
    conn: pyodbc.Connection = Depends(get_sql_connection),
    limit: int = Query(10, description="Maximum number of analytics records to return"),
    offset: int = Query(0, description="Number of analytics records to skip")
):
    cursor = conn.cursor()
    try:
        cursor.execute(f"SELECT * FROM Analytics ORDER BY ShipmentID OFFSET {offset} ROWS FETCH NEXT {limit} ROWS ONLY")
        columns = [column[0] for column in cursor.description]
        
        result = []
        for row in cursor.fetchall():
            result.append(dict(zip(columns, row)))
        
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.get("/analytics/kpi", response_model=Dict[str, Any])
async def get_kpi_metrics(conn: pyodbc.Connection = Depends(get_sql_connection)):
    cursor = conn.cursor()
    try:
        # Overall efficiency
        cursor.execute("SELECT AVG(Efficiency) as avg_efficiency FROM Analytics")
        avg_efficiency = cursor.fetchone()[0]
        
        # Average customer rating
        cursor.execute("SELECT AVG(CustomerRating) as avg_rating FROM Analytics")
        avg_rating = cursor.fetchone()[0]
        
        # On-time delivery percentage
        cursor.execute("""
            SELECT 
                COUNT(*) as total_shipments,
                SUM(CASE WHEN Delays = 0 THEN 1 ELSE 0 END) as on_time_deliveries
            FROM Analytics
        """)
        row = cursor.fetchone()
        total_shipments = row[0]
        on_time_deliveries = row[1]
        on_time_percentage = (on_time_deliveries / total_shipments) * 100 if total_shipments > 0 else 0
        
        # Shipment status breakdown
        cursor.execute("""
            SELECT Status, COUNT(*) as count
            FROM Shipments
            GROUP BY Status
        """)
        status_counts = {}
        for row in cursor.fetchall():
            status_counts[row[0]] = row[1]
        
        return {
            "avg_efficiency": avg_efficiency,
            "avg_customer_rating": avg_rating,
            "on_time_percentage": on_time_percentage,
            "status_breakdown": status_counts,
            "total_shipments": total_shipments
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

# Personnel
@app.get("/personnel", response_model=List[Personnel])
async def get_personnel(
    conn: pyodbc.Connection = Depends(get_sql_connection),
    limit: int = Query(10, description="Maximum number of personnel to return"),
    offset: int = Query(0, description="Number of personnel to skip"),
    role: Optional[str] = Query(None, description="Filter by role")
):
    cursor = conn.cursor()
    try:
        query = "SELECT * FROM Personnel"
        params = []
        
        if role:
            query += " WHERE Role = ?"
            params.append(role)
            
        query += f" ORDER BY PersonnelID OFFSET {offset} ROWS FETCH NEXT {limit} ROWS ONLY"
        
        cursor.execute(query, params)
        columns = [column[0] for column in cursor.description]
        
        result = []
        for row in cursor.fetchall():
            result.append(dict(zip(columns, row)))
        
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.get("/personnel/{personnel_id}", response_model=Personnel)
async def get_personnel_by_id(personnel_id: int, conn: pyodbc.Connection = Depends(get_sql_connection)):
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT * FROM Personnel WHERE PersonnelID = ?", (personnel_id,))
        columns = [column[0] for column in cursor.description]
        
        row = cursor.fetchone()
        if not row:
            raise HTTPException(status_code=404, detail=f"Personnel with ID {personnel_id} not found")
        
        return dict(zip(columns, row))
    except Exception as e:
        if "not found" in str(e):
            raise
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.get("/personnel/shipment/{shipment_id}", response_model=List[Personnel])
async def get_personnel_by_shipment(shipment_id: int, conn: pyodbc.Connection = Depends(get_sql_connection)):
    cursor = conn.cursor()
    try:
        query = """
            SELECT p.* 
            FROM Personnel p
            JOIN ShipmentPersonnel sp ON p.PersonnelID = sp.PersonnelID
            WHERE sp.ShipmentID = ?
        """
        
        cursor.execute(query, (shipment_id,))
        columns = [column[0] for column in cursor.description]
        
        result = []
        for row in cursor.fetchall():
            result.append(dict(zip(columns, row)))
        
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

# Neo4j Endpoints

@app.get("/graph/customer/{customer_id}")
async def get_customer_graph(
    customer_id: str, 
    depth: int = Query(2, description="Depth of relationship traversal"),
    neo4j_driver: GraphDatabase.driver = Depends(get_neo4j_driver)
):
    try:
        with neo4j_driver.session() as session:
            result = session.run(
                """
                MATCH path = (c:Customer {ID: $customer_id})-[*0..{depth}]-(n)
                RETURN path
                """,
                customer_id=customer_id,
                depth=depth
            )
            
            # Process the graph result
            nodes = []
            relationships = []
            node_ids = set()
            rel_ids = set()
            
            for record in result:
                path = record["path"]
                
                # Extract nodes
                for node in path.nodes:
                    if node.id not in node_ids:
                        node_ids.add(node.id)
                        nodes.append({
                            "id": node.id,
                            "labels": list(node.labels),
                            "properties": dict(node)
                        })
                
                # Extract relationships
                for rel in path.relationships:
                    if rel.id not in rel_ids:
                        rel_ids.add(rel.id)
                        relationships.append({
                            "id": rel.id,
                            "type": rel.type,
                            "start_node": rel.start_node.id,
                            "end_node": rel.end_node.id,
                            "properties": dict(rel)
                        })
            
            return {
                "nodes": nodes,
                "relationships": relationships
            }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Neo4j error: {str(e)}")

@app.get("/graph/shipment/{shipment_id}")
async def get_shipment_graph(
    shipment_id: str, 
    neo4j_driver: GraphDatabase.driver = Depends(get_neo4j_driver)
):
    try:
        with neo4j_driver.session() as session:
            result = session.run(
                """
                MATCH path = (s:Shipment {ID: $shipment_id})-[*0..2]-(n)
                RETURN path
                """,
                shipment_id=shipment_id
            )
            
            # Process the graph result
            nodes = []
            relationships = []
            node_ids = set()
            rel_ids = set()
            
            for record in result:
                path = record["path"]
                
                # Extract nodes
                for node in path.nodes:
                    if node.id not in node_ids:
                        node_ids.add(node.id)
                        nodes.append({
                            "id": node.id,
                            "labels": list(node.labels),
                            "properties": dict(node)
                        })
                
                # Extract relationships
                for rel in path.relationships:
                    if rel.id not in rel_ids:
                        rel_ids.add(rel.id)
                        relationships.append({
                            "id": rel.id,
                            "type": rel.type,
                            "start_node": rel.start_node.id,
                            "end_node": rel.end_node.id,
                            "properties": dict(rel)
                        })
            
            return {
                "nodes": nodes,
                "relationships": relationships
            }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Neo4j error: {str(e)}")

@app.get("/graph/delivery-chain/{parcel_id}")
async def get_delivery_chain(
    parcel_id: str, 
    neo4j_driver: GraphDatabase.driver = Depends(get_neo4j_driver)
):
    try:
        with neo4j_driver.session() as session:
            result = session.run(
                """
                MATCH path = (p:Parcel {ID: $parcel_id})-[:PART_OF]->(s:Shipment)-[:PASSED_THROUGH]->(l:Location)
                RETURN path
                ORDER BY l.timestamp
                """,
                parcel_id=parcel_id
            )
            
            # Process the graph result
            nodes = []
            relationships = []
            node_ids = set()
            rel_ids = set()
            
            for record in result:
                path = record["path"]
                
                # Extract nodes
                for node in path.nodes:
                    if node.id not in node_ids:
                        node_ids.add(node.id)
                        nodes.append({
                            "id": node.id,
                            "labels": list(node.labels),
                            "properties": dict(node)
                        })
                
                # Extract relationships
                for rel in path.relationships:
                    if rel.id not in rel_ids:
                        rel_ids.add(rel.id)
                        relationships.append({
                            "id": rel.id,
                            "type": rel.type,
                            "start_node": rel.start_node.id,
                            "end_node": rel.end_node.id,
                            "properties": dict(rel)
                        })
            
            return {
                "nodes": nodes,
                "relationships": relationships
            }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Neo4j error: {str(e)}")

@app.post("/graph/node", response_model=dict)
async def create_node(
    node_info: NodeInfo,
    neo4j_driver: GraphDatabase.driver = Depends(get_neo4j_driver)
):
    try:
        with neo4j_driver.session() as session:
            # Create a Cypher query with dynamic property assignment
            properties_str = ", ".join([f"{key}: ${key}" for key in node_info.Properties])
            
            query = f"""
            CREATE (n:{node_info.Label} {{ID: $id, {properties_str}}})
            RETURN n
            """
            
            # Prepare parameters
            params = {"id": node_info.ID, **node_info.Properties}
            
            # Execute query
            result = session.run(query, params)
            created_node = result.single()
            
            if not created_node:
                raise HTTPException(status_code=500, detail="Failed to create node")
            
            return {
                "message": f"Node with ID {node_info.ID} created successfully",
                "node": {
                    "id": created_node["n"].id,
                    "labels": list(created_node["n"].labels),
                    "properties": dict(created_node["n"])
                }
            }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Neo4j error: {str(e)}")

@app.post("/graph/relationship", response_model=dict)
async def create_relationship(
    relationship_info: RelationshipInfo,
    neo4j_driver: GraphDatabase.driver = Depends(get_neo4j_driver)
):
    try:
        with neo4j_driver.session() as session:
            # Create a Cypher query for the relationship
            properties_str = ""
            if relationship_info.Properties:
                props = ", ".join([f"{key}: ${key}" for key in relationship_info.Properties])
                properties_str = f" {{{props}}}"
            
            query = f"""
            MATCH (a), (b)
            WHERE a.ID = $start_id AND b.ID = $end_id
            CREATE (a)-[r:{relationship_info.Type}{properties_str}]->(b)
            RETURN r
            """
            
            # Prepare parameters
            params = {
                "start_id": relationship_info.StartNode,
                "end_id": relationship_info.EndNode,
                **relationship_info.Properties
            }
            
            # Execute query
            result = session.run(query, params)
            created_rel = result.single()
            
            if not created_rel:
                raise HTTPException(
                    status_code=404, 
                    detail="Could not create relationship. Make sure both nodes exist."
                )
            
            return {
                "message": f"Relationship created successfully",
                "relationship": {
                    "id": created_rel["r"].id,
                    "type": created_rel["r"].type,
                    "properties": dict(created_rel["r"])
                }
            }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Neo4j error: {str(e)}")

@app.get("/dashboard/summary", response_model=Dict[str, Any])
async def get_dashboard_summary(
    conn: pyodbc.Connection = Depends(get_sql_connection)
):
    cursor = conn.cursor()
    try:
        summary = {}
        
        # Total counts
        cursor.execute("SELECT COUNT(*) FROM Customers")
        summary["total_customers"] = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM Parcels")
        summary["total_parcels"] = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM Shipments")
        summary["total_shipments"] = cursor.fetchone()[0]
        
        # Shipment status breakdown
        cursor.execute("""
            SELECT Status, COUNT(*) as count
            FROM Shipments
            GROUP BY Status
        """)
        status_breakdown = {}
        for row in cursor.fetchall():
            status_breakdown[row[0]] = row[1]
        summary["status_breakdown"] = status_breakdown
        
        # Recent shipments
        cursor.execute("""
            SELECT TOP 5 s.ShipmentID, s.ShipmentName, s.Status, s.CurrentLocation, 
                   p.ParcelName, c.Name as CustomerName
            FROM Shipments s
            JOIN Parcels p ON s.ParcelID = p.ParcelID
            JOIN Customers c ON p.CustomerID = c.CustomerID
            ORDER BY s.ShipmentDate DESC
        """)
        columns = [column[0] for column in cursor.description]
        recent_shipments = []
        for row in cursor.fetchall():
            recent_shipments.append(dict(zip(columns, row)))
        summary["recent_shipments"] = recent_shipments
        
        # Performance metrics
        cursor.execute("""
            SELECT 
                AVG(Efficiency) as avg_efficiency,
                AVG(CustomerRating) as avg_rating,
                SUM(CASE WHEN Delays = 0 THEN 1 ELSE 0 END) as on_time_count,
                COUNT(*) as total_count
            FROM Analytics
        """)
        row = cursor.fetchone()
        summary["efficiency"] = row[0]
        summary["customer_rating"] = row[1]
        summary["on_time_percentage"] = (row[2] / row[3] * 100) if row[3] > 0 else 0
        
        return summary
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

@app.get("/dashboard/customer-insights", response_model=Dict[str, Any])
async def get_customer_insights(
    conn: pyodbc.Connection = Depends(get_sql_connection)
):
    cursor = conn.cursor()
    try:
        insights = {}
        
        # Customer type distribution
        cursor.execute("""
            SELECT Type, COUNT(*) as count
            FROM Customers
            GROUP BY Type
        """)
        type_distribution = {}
        for row in cursor.fetchall():
            type_distribution[row[0]] = row[1]
        insights["customer_types"] = type_distribution
        
        # Top customers by shipment volume
        cursor.execute("""
            SELECT TOP 5 c.CustomerID, c.Name, COUNT(s.ShipmentID) as shipment_count
            FROM Customers c
            JOIN Parcels p ON c.CustomerID = p.CustomerID
            JOIN Shipments s ON p.ParcelID = s.ParcelID
            GROUP BY c.CustomerID, c.Name
            ORDER BY shipment_count DESC
        """)
        columns = [column[0] for column in cursor.description]
        top_customers = []
        for row in cursor.fetchall():
            top_customers.append(dict(zip(columns, row)))
        insights["top_customers"] = top_customers
        
        # Customer satisfaction by type
        cursor.execute("""
            SELECT c.Type, AVG(a.CustomerRating) as avg_rating
            FROM Customers c
            JOIN Parcels p ON c.CustomerID = p.CustomerID
            JOIN Shipments s ON p.ParcelID = s.ParcelID
            JOIN Analytics a ON s.ShipmentID = a.ShipmentID
            GROUP BY c.Type
        """)
        satisfaction_by_type = {}
        for row in cursor.fetchall():
            satisfaction_by_type[row[0]] = row[1]
        insights["satisfaction_by_type"] = satisfaction_by_type
        
        return insights
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database error: {str(e)}")
    finally:
        cursor.close()

# Run the app
if "get_ipython" in globals():
    # For Jupyter Notebook
    import nest_asyncio
    nest_asyncio.apply()
    from uvicorn import Config, Server
    config = Config(app=app, host="0.0.0.0", port=8000, log_level="info")
    server = Server(config)
    import asyncio
    loop = asyncio.get_event_loop()
    loop.create_task(server.serve())